## Combine Files
The save_info script outputs 1 file per each input file. Here, we concatenate across those files to give just one output file.

In [1]:
import xarray as xr
import os
import fnmatch
import re
from tqdm.notebook import tqdm
print(os.getcwd())

/home/users/jamd1/Isca/jobs/cesm/3_hour/hottest_quant


In [2]:
os.path.join(os.getcwd(), 'pre_industrial/REFHT_quant50')

'/home/users/jamd1/Isca/jobs/cesm/3_hour/hottest_quant/pre_industrial/REFHT_quant50'

In [ ]:
output_dir = '/Users/joshduffield/Desktop/output/'
output_dir = os.path.join(os.getcwd(), 'pre_industrial/REFHT_quant50')
out_files_all = os.listdir(output_dir)
# only keep files of correct format
out_files_all = [file for file in out_files_all if
                  re.fullmatch(r"output\d+\.nc", file)]     # must match output{any_number}.nc
n_out_files = len(out_files_all)
print(n_out_files)
ds = []
for i in tqdm(range(n_out_files)):
    ds.append(xr.load_dataset(os.path.join(output_dir, out_files_all[i])))
ds = xr.concat(ds, dim="file")        # concat all files over new dimension
ds = ds.drop_dims("time")             # change time to a variable, and remove dimension
ds = ds.reset_coords("time")
ds = ds.max(dim='file')    # each value has only one non nan value across all files, so take max to only keep non nan values

40


  0%|          | 0/40 [00:00<?, ?it/s]

In [52]:
out_file = os.path.join(output_dir, 'output.nc')
if os.path.exists(out_file):
    raise ValueError('Output file already exists at {}'.format(out_file))
complevel = 4
ds.to_netcdf(out_file, format="NETCDF4",
             encoding={var: {"zlib": True, "complevel": complevel} for var in ds.data_vars})